# Create a training & validation dataset

In [2]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset("frenchtext/nlptextdoc-2023_05-creditmutuel", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/34364 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /models/huggingface/datasets/frenchtext___parquet/frenchtext--nlptextdoc-2023_05-creditmutuel-1698ce1d76ab5419/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


In [3]:
dataset

Dataset({
    features: ['SiteIndex', 'DocId', 'Url', 'Text', 'Words', 'Lang'],
    num_rows: 34364
})

In [5]:
dataset[0]["Text"]

'******\nMentions légales - Crédit Mutuel\n======\n\nDevenir client du Crédit Mutuel\n\n# Editeur\n\nConfédération Nationale du Crédit\xa0Mutuel\n\n- Siège social\xa0: 88-90 rue Cardinet 750017 Paris\n- Téléphone\xa0: 01\xa044\xa001\xa010\xa010\n- Représentant légal\xa0: M. Pierre-Edouard BATARD\n- Directeur de la publication\xa0: M. Pierre-Edouard BATARD\n\n# Hébergeur\n\n- Siège social\xa0: 4 rue Frédéric-Guillaume Raiffeisen - 67000\xa0Strasbourg\n- Immatriculé au RCS Strasbourg B\xa0312\xa0730\xa0674\n- Téléphone\xa0: 03\xa088\xa014\xa088\xa014\n\nCe service de communication audiovisuelle a fait l’objet d’une déclaration auprès du Parquet du Tribunal de Grande Instance de STRASBOURG le 08/02/1999 sous la référence L10 2547/99 et du Conseil Supérieur de l’Audiovisuel récépissé en date du 17/02/1999. Dans le cadre des présentes, les 15 groupes de Crédit\xa0Mutuel commercialisant les produits Crédit\xa0Mutuel, sont sans distinction dénommés ci-après «\xa0le Crédit\xa0Mutuel\xa0».\n\nL

In [ ]:
from random import randint
from itertools import chain
from functools import partial


# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample


# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# print random sample
print(dataset[randint(0, len(dataset))]["text"])

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

In [ ]:
training_input_path = f's3://{sess.default_bucket()}/processed/llama/dolly/train'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")